In [5]:
import pandas as pd

tebu_glcm = pd.read_csv('tebu_glcm.csv')
tebu_sen1_chirps_terrain = pd.read_csv('tebu_sen1_chirps_terrain.csv')
tebu_sen2 = pd.read_csv('tebu_sen2.csv')

padi_glcm = pd.read_csv('padi_glcm.csv')
padi_sen1_chirps_terrain = pd.read_csv('padi_sen1_chirps_terrain.csv')
padi_sen2 = pd.read_csv('padi_sen2.csv')

jagung_glcm = pd.read_csv('jagung_glcm.csv')
jagung_sen1_chirps_terrain = pd.read_csv('jagung_sen1_chirps_terrain.csv')
jagung_sen2 = pd.read_csv('jagung_sen2.csv')

others_glcm = pd.read_csv('others_glcm.csv')
others_sen1_chirps_terrain = pd.read_csv('others_sen1_chirps_terrain.csv')
others_sen2 = pd.read_csv('others_sen2.csv')

## **PROCESS DATA**

In [6]:
def process_sentinel2(file_path):
    """Memproses data Sentinel-2: membersihkan, merename, dan pivoting berdasarkan 'name' dan 'date'."""
    df = pd.read_csv(file_path)
    df = df.drop(columns=['system:index', 'end_date', 'time', '.geo'], errors='ignore')
    df = df.rename(columns=lambda x: x.replace('_median', ''))
    df['start_date'] = pd.to_datetime(df['start_date']).dt.strftime('%Y-%m')
    df = df[(df['start_date'] >= '2024-01') & (df['start_date'] <= '2024-12')]

    # Pivot data
    df_pivot = df.pivot(index='name', columns='start_date')
    df_pivot.columns = [f"{col[0]}_{col[1]}" for col in df_pivot.columns]
    df_pivot.reset_index(inplace=True)

    return df_pivot

def process_sentinel1_chirps_terrain(file_path):
    """Memproses data Sentinel-1 + CHIRPS + Terrain: membersihkan dan pivoting berdasarkan 'Location' dan 'start_date'."""
    df = pd.read_csv(file_path)
    selected_columns = ['Location', 'start_date', 'VV', 'VH', 'CHIRPS', 'Aspect', 'Elevation', 'Slope']
    df = df[selected_columns]

    # Pivot data
    df_pivot = df.pivot(index='Location', columns='start_date')
    df_pivot.columns = [f"{col[0]}_{col[1]}" for col in df_pivot.columns]
    df_pivot.reset_index(inplace=True)

    return df_pivot

def process_glcm(file_path):
    """Memproses data GLCM: membersihkan dan pivoting berdasarkan 'location' dan 'date'."""
    df = pd.read_csv(file_path)
    df = df.drop(columns=['system:index', '.geo'], errors='ignore')

    # Pivot data
    df_pivot = df.pivot(index='location', columns='date')
    df_pivot.columns = [f"{col[0]}_{col[1]}" for col in df_pivot.columns]
    df_pivot.reset_index(inplace=True)

    return df_pivot

def merge_datasets(sen2_df, sen1_df, glcm_df, crop_name):
    """Menggabungkan data Sentinel-2, Sentinel-1 + CHIRPS + Terrain, dan GLCM menjadi satu dataset."""

    merged_df = pd.merge(sen2_df, sen1_df, left_on='name', right_on='Location', how='inner')
    merged_df = pd.merge(merged_df, glcm_df, left_on='Location', right_on='location', how='inner')
    merged_df = merged_df.drop(columns=['Location', 'name'])

    merged_df = merged_df[['location'] + [col for col in merged_df.columns if col != 'location']]
    merged_df['class'] = crop_name  # Misalnya 'Tebu', 'Padi', atau 'Jagung'

    return merged_df

# **NaN HANDLING**

## **Tebu**

In [7]:
tebu_sen2 = process_sentinel2('tebu_sen2.csv')
tebu_sen1 = process_sentinel1_chirps_terrain('tebu_sen1_chirps_terrain.csv')
tebu_glcm = process_glcm('tebu_glcm.csv')

### SEN2

In [8]:
tebu_sen2

,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12
0,Location_1,375.5,652.5,1808.0,4173.0,434.5,523.5,808.5,621.5,591.5,...,4752.5,4941.5,4593.5,3520.5,3360.0,1685.5,2004.0,2519.5,3610.0,4579.0
1,Location_10,413.0,580.0,318.5,445.0,490.0,261.0,487.5,471.0,548.5,...,3597.0,2756.0,3819.5,3630.0,3926.0,3649.5,2256.0,3565.5,4834.5,4923.0
2,Location_100,877.0,484.5,723.0,618.0,484.0,1515.0,513.0,744.0,432.0,...,4753.0,2237.0,3664.5,4073.5,1080.0,3209.0,1115.0,3034.0,2895.5,5722.0
3,Location_101,1067.0,646.0,2348.0,445.0,1026.5,694.0,733.5,1995.0,793.0,...,4439.0,3174.0,2492.5,1978.0,1883.5,3075.0,2887.0,3616.0,4303.5,4269.0
4,Location_102,1614.0,782.0,497.5,349.0,598.0,512.0,584.0,810.0,474.0,...,4747.0,4948.0,4012.0,3858.0,4208.5,4089.0,4086.5,2419.0,2823.0,3758.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Location_95,1201.0,1194.0,1405.0,1052.0,1881.5,543.0,786.0,1836.5,726.0,...,3260.0,4068.0,4461.0,3482.0,2177.0,2298.0,2734.0,3205.0,4814.0,3837.0
137,Location_96,4004.0,735.5,803.5,465.0,396.0,545.0,793.0,577.0,561.0,...,4134.5,4216.0,2514.0,3203.0,3765.0,3469.0,3356.0,1901.0,2428.0,2984.0
138,Location_97,2268.0,710.0,2063.0,854.0,807.5,722.0,844.0,2110.0,642.0,...,4093.0,4552.0,3758.5,3064.0,4285.5,4502.0,3623.0,3344.0,3623.0,3507.0
139,Location_98,973.0,681.0,1790.0,NaN,759.0,776.5,1869.5,775.0,1188.5,...,4896.0,NaN,4148.5,3650.0,3106.5,3118.0,4420.0,3231.0,2733.0,4901.0


In [9]:
tebu_sen2[tebu_sen2.isna().any(axis=1)]

,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12
5,Location_103,1434.0,517.5,542.0,NaN,988.0,1024.0,1804.0,836.5,1948.5,...,2898.0,NaN,3433.0,2858.5,2790.0,2853.0,3220.0,3066.0,4587.0,3724.0
6,Location_104,971.0,572.5,1907.0,NaN,708.0,631.0,609.5,3298.0,793.0,...,4573.0,NaN,3671.0,3339.0,2407.5,4339.0,2603.0,2782.5,3099.0,5608.0
7,Location_105,1643.0,372.5,324.5,293.0,649.0,459.0,911.0,1216.0,898.0,...,3171.0,3989.5,4657.5,3575.5,3493.0,3051.5,3206.5,1577.5,2291.5,NaN
13,Location_110,1525.0,501.0,1123.5,NaN,590.0,528.0,837.0,579.0,633.0,...,3725.5,NaN,3525.0,3818.0,1967.0,1674.5,3707.5,3575.0,2567.0,4976.0
15,Location_112,896.0,493.0,461.0,NaN,595.0,753.5,719.0,933.0,733.0,...,4503.0,NaN,3751.5,4004.0,3976.0,5119.0,3443.0,2417.5,4744.5,5287.0
16,Location_113,1530.0,376.5,280.5,250.0,751.0,375.5,710.5,449.5,524.5,...,4399.5,4498.0,4326.5,4415.0,4338.5,4042.5,3880.5,3380.5,3412.0,NaN
17,Location_114,993.0,828.0,826.0,753.5,527.0,586.0,612.0,580.5,712.5,...,3288.0,3287.0,2598.0,2426.0,2062.5,2073.5,2507.0,3165.5,3214.5,NaN
18,Location_115,1554.0,431.5,343.0,265.0,2340.0,2838.0,557.0,695.5,784.5,...,4086.5,4134.5,4861.0,4795.0,3989.0,3141.5,3300.0,1502.5,2316.0,NaN
20,Location_117,1583.0,920.5,1361.0,329.0,3862.0,371.0,463.5,450.5,741.5,...,4614.0,3287.0,5762.5,2739.5,3234.0,2844.5,3209.5,3070.0,2976.0,NaN
21,Location_118,1514.5,416.0,600.5,262.5,354.0,492.0,890.5,584.5,568.5,...,4677.5,4332.0,3983.0,4106.5,4031.5,3752.0,3800.5,1979.0,2761.0,NaN


In [10]:
def fill_nan_by_group(df):
    # Buat dictionary untuk mengelompokkan kolom berdasarkan prefix (misalnya 'b1', 'b2', ...)
    grouped_columns = {}

    for col in df.columns:
        prefix = ''.join(filter(str.isalpha, col))  # Ambil huruf dari nama kolom (misal 'b1' dari 'b1_2024-01')
        if prefix not in grouped_columns:
            grouped_columns[prefix] = []
        grouped_columns[prefix].append(col)

    # Iterasi setiap grup untuk melakukan pengisian NaN
    for group, cols in grouped_columns.items():
        df[cols] = df[cols].apply(lambda series: series.interpolate(method='linear', limit_direction='both'), axis=0)

    return df

# Terapkan ke dataset
tebu_sen2 = fill_nan_by_group(tebu_sen2)

# Tampilkan baris yang masih memiliki NaN (jika ada)
tebu_sen2[tebu_sen2.isna().any(axis=1)]


<ipython-input-10-0f6f2f8ca1cb>:13: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[cols] = df[cols].apply(lambda series: series.interpolate(method='linear', limit_direction='both'), axis=0)


,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12


In [11]:
tebu_sen2

,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12
0,Location_1,375.5,652.5,1808.0,4173.0,434.5,523.5,808.5,621.5,591.5,...,4752.5,4941.5,4593.5,3520.5,3360.0,1685.5,2004.0,2519.5,3610.0,4579.0
1,Location_10,413.0,580.0,318.5,445.0,490.0,261.0,487.5,471.0,548.5,...,3597.0,2756.0,3819.5,3630.0,3926.0,3649.5,2256.0,3565.5,4834.5,4923.0
2,Location_100,877.0,484.5,723.0,618.0,484.0,1515.0,513.0,744.0,432.0,...,4753.0,2237.0,3664.5,4073.5,1080.0,3209.0,1115.0,3034.0,2895.5,5722.0
3,Location_101,1067.0,646.0,2348.0,445.0,1026.5,694.0,733.5,1995.0,793.0,...,4439.0,3174.0,2492.5,1978.0,1883.5,3075.0,2887.0,3616.0,4303.5,4269.0
4,Location_102,1614.0,782.0,497.5,349.0,598.0,512.0,584.0,810.0,474.0,...,4747.0,4948.0,4012.0,3858.0,4208.5,4089.0,4086.5,2419.0,2823.0,3758.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Location_95,1201.0,1194.0,1405.0,1052.0,1881.5,543.0,786.0,1836.5,726.0,...,3260.0,4068.0,4461.0,3482.0,2177.0,2298.0,2734.0,3205.0,4814.0,3837.0
137,Location_96,4004.0,735.5,803.5,465.0,396.0,545.0,793.0,577.0,561.0,...,4134.5,4216.0,2514.0,3203.0,3765.0,3469.0,3356.0,1901.0,2428.0,2984.0
138,Location_97,2268.0,710.0,2063.0,854.0,807.5,722.0,844.0,2110.0,642.0,...,4093.0,4552.0,3758.5,3064.0,4285.5,4502.0,3623.0,3344.0,3623.0,3507.0
139,Location_98,973.0,681.0,1790.0,718.0,759.0,776.5,1869.5,775.0,1188.5,...,4896.0,4379.5,4148.5,3650.0,3106.5,3118.0,4420.0,3231.0,2733.0,4901.0


### SEN 1 ETC

In [12]:
tebu_sen1

,Location,VV_2024-01,VV_2024-02,VV_2024-03,VV_2024-04,VV_2024-05,VV_2024-06,VV_2024-07,VV_2024-08,VV_2024-09,...,Slope_2024-04,Slope_2024-05,Slope_2024-06,Slope_2024-07,Slope_2024-08,Slope_2024-09,Slope_2024-10,Slope_2024-11,Slope_2024-12,Slope_2025-01
0,Location_1,-7.528994,-8.358209,-7.823781,-7.090571,-9.096493,-10.259257,-7.526961,-10.677412,-7.863123,...,1.275279,1.275279,1.275279,1.275279,1.275279,1.275279,1.275279,1.275279,1.275279,1.275279
1,Location_10,-8.386272,-8.175400,-12.774248,-11.361732,-11.603675,-10.056092,-9.088675,-9.418470,-9.694802,...,0.692725,0.692725,0.692725,0.692725,0.692725,0.692725,0.692725,0.692725,0.692725,0.692725
2,Location_100,-10.352246,-13.229273,-14.460985,-11.122012,-12.017736,-14.072794,-15.638367,-12.799426,-14.656122,...,1.122856,1.122856,1.122856,1.122856,1.122856,1.122856,1.122856,1.122856,1.122856,1.122856
3,Location_101,-10.504789,-11.420958,-10.219746,-13.046667,-10.568393,-9.942958,-9.945644,-12.687621,-10.862214,...,2.251384,2.251384,2.251384,2.251384,2.251384,2.251384,2.251384,2.251384,2.251384,2.251384
4,Location_102,-11.153065,-8.972543,-7.992522,-9.984353,-9.854440,-8.840909,-9.032706,-9.371288,-9.684007,...,1.385274,1.385274,1.385274,1.385274,1.385274,1.385274,1.385274,1.385274,1.385274,1.385274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Location_95,-10.108644,-6.024963,-9.309017,-5.818107,-6.515406,-7.979998,-11.913324,-12.159817,-10.739272,...,3.297811,3.297811,3.297811,3.297811,3.297811,3.297811,3.297811,3.297811,3.297811,3.297811
137,Location_96,-9.298239,-8.199208,-10.501211,-12.834477,-12.077189,-11.110977,-10.026290,-11.367002,-17.092108,...,2.643056,2.643056,2.643056,2.643056,2.643056,2.643056,2.643056,2.643056,2.643056,2.643056
138,Location_97,-10.690202,-11.444712,-9.435060,-6.604188,-10.597640,-8.527624,-10.366847,-11.742486,-10.209420,...,5.386383,5.386383,5.386383,5.386383,5.386383,5.386383,5.386383,5.386383,5.386383,5.386383
139,Location_98,-7.943053,-10.810380,-9.475478,-12.311799,-11.336309,-8.041310,-11.879617,-11.400428,-12.220589,...,0.692770,0.692770,0.692770,0.692770,0.692770,0.692770,0.692770,0.692770,0.692770,0.692770


In [13]:
tebu_sen1[tebu_sen1.isna().any(axis=1)]

,Location,VV_2024-01,VV_2024-02,VV_2024-03,VV_2024-04,VV_2024-05,VV_2024-06,VV_2024-07,VV_2024-08,VV_2024-09,...,Slope_2024-04,Slope_2024-05,Slope_2024-06,Slope_2024-07,Slope_2024-08,Slope_2024-09,Slope_2024-10,Slope_2024-11,Slope_2024-12,Slope_2025-01


### GLCM

In [14]:
tebu_glcm

,location,VH_diss_2024-01,VH_diss_2024-02,VH_diss_2024-03,VH_diss_2024-04,VH_diss_2024-05,VH_diss_2024-06,VH_diss_2024-07,VH_diss_2024-08,VH_diss_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01
0,Location_1,14.762538,14.621534,15.387462,15.330795,14.728182,15.441591,16.767409,18.888136,17.079182,...,5.238523,5.266343,5.249825,5.247753,5.236627,5.261103,5.271101,5.261828,5.243223,5.252348
1,Location_10,15.712576,15.767443,16.014242,16.497273,16.498750,15.239318,16.138455,15.868545,18.910136,...,5.279375,5.290195,5.262574,5.261156,5.245557,5.273984,5.253889,5.266538,5.273718,5.274358
2,Location_100,16.297083,14.812216,15.167197,15.525170,14.678220,14.877773,15.837818,15.833545,14.975364,...,5.262306,5.260171,5.252504,5.254760,5.281956,5.247375,5.267330,5.235875,5.291425,5.269675
3,Location_101,17.189432,18.705795,16.941515,17.254261,17.709697,17.595591,16.635818,17.298227,17.421545,...,5.290820,5.281692,5.280640,5.274906,5.293938,5.282001,5.287665,5.255166,5.283386,5.272209
4,Location_102,15.944205,17.226023,15.947311,16.695227,16.322159,16.728136,16.056636,15.093500,15.860591,...,5.279391,5.285950,5.269042,5.267692,5.256470,5.264337,5.274964,5.274470,5.271800,5.290775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Location_95,17.349924,15.721364,15.018485,14.649148,14.686364,15.601818,17.556091,16.810136,15.878773,...,5.261040,5.251318,5.253960,5.272335,5.257065,5.250710,5.253939,5.252784,5.259103,5.273824
137,Location_96,16.688409,17.372386,17.130303,17.787386,17.796591,16.630045,17.814864,16.855727,16.290955,...,5.284255,5.278998,5.277447,5.272930,5.256900,5.265709,5.277344,5.257590,5.273260,5.270668
138,Location_97,17.192197,16.355966,17.210303,17.336534,16.558674,17.134091,17.430864,17.828773,17.584591,...,5.269207,5.280895,5.261339,5.268494,5.273131,5.255169,5.258499,5.274448,5.261112,5.282638
139,Location_98,14.979735,14.737386,15.275492,14.868295,14.850682,14.968273,15.316727,15.950364,15.132636,...,5.243550,5.264489,5.255463,5.259280,5.273817,5.243448,5.231638,5.226259,5.256745,5.243032


In [15]:
tebu_glcm[tebu_glcm.isna().any(axis=1)]

,location,VH_diss_2024-01,VH_diss_2024-02,VH_diss_2024-03,VH_diss_2024-04,VH_diss_2024-05,VH_diss_2024-06,VH_diss_2024-07,VH_diss_2024-08,VH_diss_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01


## **Padi**

In [16]:
padi_sen2 = process_sentinel2('padi_sen2.csv')
padi_sen1 = process_sentinel1_chirps_terrain('padi_sen1_chirps_terrain.csv')
padi_glcm = process_glcm('padi_glcm.csv')

### SEN2

In [17]:
padi_sen2

,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12
0,Location_1,1022.0,548.0,245.5,601.0,502.5,547.0,492.0,379.0,659.0,...,3542.5,4535.0,2521.0,1584.0,2368.5,2607.0,3135.0,3738.0,2376.0,3888.0
1,Location_10,235.5,581.5,367.0,107.0,205.0,306.5,292.0,316.0,425.0,...,3958.5,3744.0,3591.0,3579.0,3500.5,3502.5,3597.0,4309.0,4263.0,3956.0
2,Location_100,5496.5,3543.0,452.0,488.0,545.0,432.0,443.0,667.0,736.5,...,291.0,1839.0,2009.0,3579.0,3594.0,2711.5,2976.0,1894.0,2801.0,3762.0
3,Location_101,5505.5,2093.0,712.0,513.0,515.0,426.0,447.0,619.0,601.5,...,1187.0,1988.5,1908.0,4068.0,3885.5,3202.0,2736.0,2476.0,2740.0,4320.0
4,Location_102,4667.0,2066.5,3955.0,525.0,423.5,416.5,665.0,771.0,779.0,...,5101.0,890.0,1718.0,3282.0,2742.5,3295.0,3171.5,1503.0,1608.0,3541.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Location_95,3699.0,332.5,1401.0,592.5,495.5,478.5,572.5,746.0,797.5,...,3740.0,2796.5,2883.5,2434.0,3562.0,2781.0,1799.5,1604.5,1804.0,2965.0
356,Location_96,5650.0,556.0,793.0,337.0,355.5,392.0,551.0,701.0,551.0,...,1974.0,733.5,2334.5,3609.0,2292.0,2530.0,3727.0,2143.0,930.0,2250.0
357,Location_97,5286.5,407.0,818.0,481.0,410.5,429.0,571.5,734.0,734.0,...,1973.0,2100.0,2307.5,2772.0,2383.5,2461.0,1893.5,1533.5,1890.0,2639.0
358,Location_98,3289.0,445.0,639.0,642.5,417.0,514.0,620.5,569.5,521.5,...,2063.0,1536.5,2946.5,3438.5,2653.5,3016.0,3237.5,2117.0,1741.0,NaN


In [18]:
padi_sen2[padi_sen2.isna().any(axis=1)]

,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12
25,Location_121,1947.5,1417.5,391.0,440.5,359.5,430.0,531.0,611.0,633.5,...,1165.0,1130.0,2530.5,1873.5,1763.0,1712.0,1720.5,2264.0,NaN,3243.5
28,Location_124,4883.5,1781.0,NaN,390.5,321.5,447.0,545.0,551.5,532.0,...,NaN,1637.5,3294.0,2733.5,2149.5,3683.5,3454.0,2374.0,2280.5,2724.0
46,Location_140,6580.0,2253.5,NaN,504.0,487.0,442.0,300.0,417.0,780.0,...,NaN,2451.0,1575.0,2397.5,3359.0,3124.0,3073.0,3751.5,4385.0,1680.5
47,Location_141,4429.5,2239.5,NaN,499.0,501.0,395.0,254.0,403.0,729.0,...,NaN,1837.0,1531.5,2150.0,3721.0,3244.5,3217.0,2899.0,2526.5,2058.0
48,Location_142,1820.0,857.0,1044.0,1277.5,870.0,804.0,2957.0,931.0,NaN,...,871.0,1164.0,647.0,495.0,2074.0,607.0,NaN,1105.0,177.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,Location_80,4388.5,482.0,NaN,493.5,349.5,415.0,480.0,415.0,574.0,...,NaN,2270.5,2690.5,3062.5,2993.0,4952.5,3192.0,1627.0,1823.0,2921.0
350,Location_90,3935.0,2295.0,NaN,2569.5,669.5,792.0,589.5,709.5,783.0,...,NaN,3897.5,1717.5,2246.5,2455.0,3165.5,3068.5,3322.0,2401.0,1809.0
351,Location_91,4899.5,427.0,NaN,621.5,308.5,441.0,531.0,678.0,615.0,...,NaN,1722.5,2529.5,3820.0,2509.0,2368.0,1539.5,1530.5,2189.0,3904.0
354,Location_94,3974.5,880.0,681.0,736.0,567.5,620.0,367.5,394.0,460.0,...,1447.0,2212.0,1602.5,1911.0,3907.0,4000.0,2746.0,1610.0,2748.0,NaN


In [19]:
def fill_nan_by_group(df):
    # Buat dictionary untuk mengelompokkan kolom berdasarkan prefix (misalnya 'b1', 'b2', ...)
    grouped_columns = {}

    for col in df.columns:
        prefix = ''.join(filter(str.isalpha, col))  # Ambil huruf dari nama kolom (misal 'b1' dari 'b1_2024-01')
        if prefix not in grouped_columns:
            grouped_columns[prefix] = []
        grouped_columns[prefix].append(col)

    # Iterasi setiap grup untuk melakukan pengisian NaN
    for group, cols in grouped_columns.items():
        df[cols] = df[cols].apply(lambda series: series.interpolate(method='linear', limit_direction='both'), axis=0)

    return df

# Terapkan ke dataset
padi_sen2 = fill_nan_by_group(padi_sen2)

# Tampilkan baris yang masih memiliki NaN (jika ada)
padi_sen2[padi_sen2.isna().any(axis=1)]

<ipython-input-19-1bb6a9b8f4b6>:13: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[cols] = df[cols].apply(lambda series: series.interpolate(method='linear', limit_direction='both'), axis=0)


,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12


### SEN 1 ETC

In [20]:
padi_sen1

,Location,VV_2024-01,VV_2024-02,VV_2024-03,VV_2024-04,VV_2024-05,VV_2024-06,VV_2024-07,VV_2024-08,VV_2024-09,...,Slope_2024-04,Slope_2024-05,Slope_2024-06,Slope_2024-07,Slope_2024-08,Slope_2024-09,Slope_2024-10,Slope_2024-11,Slope_2024-12,Slope_2025-01
0,Location_1,-9.369951,-8.822844,-10.618818,-12.062128,-9.682688,-8.703713,-11.574151,-13.000433,-12.497883,...,0.618303,0.618303,0.618303,0.618303,0.618303,0.618303,0.618303,0.618303,0.618303,0.618303
1,Location_10,-7.945746,-9.893126,-9.431407,-8.503016,-8.896021,-8.566622,-8.505093,-9.090603,-7.681518,...,1.318209,1.318209,1.318209,1.318209,1.318209,1.318209,1.318209,1.318209,1.318209,1.318209
2,Location_100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Location_101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Location_102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Location_95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,Location_97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,Location_98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
padi_sen1[padi_sen1.isna().any(axis=1)]

,Location,VV_2024-01,VV_2024-02,VV_2024-03,VV_2024-04,VV_2024-05,VV_2024-06,VV_2024-07,VV_2024-08,VV_2024-09,...,Slope_2024-04,Slope_2024-05,Slope_2024-06,Slope_2024-07,Slope_2024-08,Slope_2024-09,Slope_2024-10,Slope_2024-11,Slope_2024-12,Slope_2025-01
2,Location_100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Location_101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Location_102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Location_103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Location_104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Location_95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,Location_97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,Location_98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### GLCM

In [22]:
padi_glcm

,location,VH_diss_2024-01,VH_diss_2024-02,VH_diss_2024-03,VH_diss_2024-04,VH_diss_2024-05,VH_diss_2024-06,VH_diss_2024-07,VH_diss_2024-08,VH_diss_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01
0,Location_1,19.580152,20.973409,15.391061,14.857330,16.601515,17.604545,16.154773,16.560818,16.934955,...,5.237976,5.267448,5.236203,5.252861,5.254998,5.252413,5.253070,5.230378,5.232962,5.284922
1,Location_10,22.606591,22.159432,23.944886,19.715795,22.526894,20.422864,20.339818,19.990455,23.371409,...,5.239840,5.257500,5.255708,5.253817,5.250058,5.255002,5.229077,5.261958,5.261547,5.252367
2,Location_100,15.996553,27.953466,28.982917,27.941894,25.198136,19.018295,16.668371,18.164886,20.254470,...,5.287934,5.298460,5.257171,5.261669,5.272243,5.275870,5.279690,5.267056,5.273107,5.264176
3,Location_101,21.652841,25.881477,28.145758,30.161818,27.809318,22.904773,22.235114,23.202727,23.641742,...,5.314700,5.300666,5.282329,5.283011,5.285661,5.296321,5.311079,5.309027,5.311099,5.310650
4,Location_102,17.485530,18.780227,23.843902,27.117121,20.473682,16.092955,18.547992,22.622898,23.283485,...,5.298946,5.278748,5.271373,5.272000,5.271573,5.283117,5.300721,5.292780,5.273093,5.288397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Location_95,21.111288,17.175625,19.060227,23.788864,19.855500,21.114318,18.994053,22.661080,21.603409,...,5.287816,5.279820,5.290793,5.285480,5.286758,5.281129,5.283475,5.295009,5.292830,5.290518
356,Location_96,19.320492,19.839830,22.124356,31.458409,23.543591,17.237500,18.555189,22.052443,17.498106,...,5.295719,5.280183,5.238085,5.259750,5.243627,5.284490,5.261643,5.291534,5.281970,5.267442
357,Location_97,18.757765,18.073466,22.278674,30.306515,21.843500,16.939886,18.979053,19.942330,19.391136,...,5.303066,5.265829,5.262952,5.259913,5.264276,5.277587,5.243198,5.293644,5.272130,5.263143
358,Location_98,19.879659,16.704773,18.752424,23.709167,20.594045,16.868750,18.572614,19.657614,22.013939,...,5.306455,5.296970,5.288000,5.284948,5.278363,5.279864,5.295619,5.288298,5.276369,5.294733


In [23]:
padi_glcm[padi_glcm.isna().any(axis=1)]

,location,VH_diss_2024-01,VH_diss_2024-02,VH_diss_2024-03,VH_diss_2024-04,VH_diss_2024-05,VH_diss_2024-06,VH_diss_2024-07,VH_diss_2024-08,VH_diss_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01
48,Location_142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **Jagung**

In [24]:
jagung_sen2 = process_sentinel2('jagung_sen2.csv')
jagung_sen1 = process_sentinel1_chirps_terrain('jagung_sen1_chirps_terrain.csv')
jagung_glcm = process_glcm('jagung_glcm.csv')

### SEN2

In [25]:
jagung_sen2

,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12
0,Location_1,344.0,799.0,548.0,1606.0,382.5,284.0,477.0,315.0,1117.0,...,3630.0,2319.0,2480.0,2875.0,3570.0,3315.0,3779.5,3116.0,4151.0,1629.0
1,Location_10,1271.0,503.5,294.0,313.0,1189.0,587.0,407.0,525.0,765.5,...,2814.0,3092.0,2516.0,2238.0,3624.0,3950.0,2209.5,3235.5,4220.0,NaN
2,Location_100,770.0,463.5,388.5,923.0,895.5,414.0,601.0,458.0,503.0,...,3629.5,3012.0,4081.5,3508.0,3399.0,3298.0,3782.5,3913.0,3808.5,4285.0
3,Location_101,1337.0,738.5,415.5,623.5,712.0,370.0,438.0,715.0,847.0,...,2960.0,3718.0,2804.0,4075.0,4192.5,3368.0,3170.0,4188.5,4395.0,1041.0
4,Location_102,860.0,459.0,491.0,945.0,321.0,384.0,499.0,419.0,422.0,...,4705.0,4994.5,1680.0,3952.0,4187.0,3696.0,3882.0,3694.0,4287.0,4536.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,Location_95,494.0,352.5,264.5,655.5,711.0,1385.0,1344.0,1182.5,1085.5,...,3111.0,3005.0,2331.5,3290.0,3625.0,2708.0,3005.0,2822.0,2776.0,2660.0
312,Location_96,1637.0,346.0,198.5,422.0,595.0,784.0,768.0,714.0,1048.5,...,3148.5,2969.0,2092.0,2391.0,2942.0,3675.0,3949.5,2736.0,3315.5,2765.5
313,Location_97,1848.0,673.5,606.0,811.0,1021.0,655.0,644.0,677.5,1296.0,...,2993.5,4234.0,3355.0,2433.0,3615.0,3374.5,3037.0,3860.0,3452.0,4387.0
314,Location_98,4805.0,627.0,442.5,1901.5,483.0,351.5,611.5,564.5,788.0,...,4422.5,3923.0,3209.0,3999.0,2990.0,2807.0,3264.0,4821.5,4143.5,2984.0


In [26]:
jagung_sen2[padi_sen2.isna().any(axis=1)]

<ipython-input-26-0e7686324092>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  jagung_sen2[padi_sen2.isna().any(axis=1)]


,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12


In [27]:
def fill_nan_by_group(df):
    # Buat dictionary untuk mengelompokkan kolom berdasarkan prefix (misalnya 'b1', 'b2', ...)
    grouped_columns = {}

    for col in df.columns:
        prefix = ''.join(filter(str.isalpha, col))  # Ambil huruf dari nama kolom (misal 'b1' dari 'b1_2024-01')
        if prefix not in grouped_columns:
            grouped_columns[prefix] = []
        grouped_columns[prefix].append(col)

    # Iterasi setiap grup untuk melakukan pengisian NaN
    for group, cols in grouped_columns.items():
        df[cols] = df[cols].apply(lambda series: series.interpolate(method='linear', limit_direction='both'), axis=0)

    return df

# Terapkan ke dataset
jagung_sen2 = fill_nan_by_group(jagung_sen2)

# Tampilkan baris yang masih memiliki NaN (jika ada)
jagung_sen2[jagung_sen2.isna().any(axis=1)]

<ipython-input-27-bbd789bb85ea>:13: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[cols] = df[cols].apply(lambda series: series.interpolate(method='linear', limit_direction='both'), axis=0)


,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12


### SEN 1 ETC

In [28]:
jagung_sen2

,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12
0,Location_1,344.0,799.0,548.0,1606.0,382.5,284.0,477.0,315.0,1117.0,...,3630.0,2319.0,2480.0,2875.0,3570.0,3315.0,3779.5,3116.0,4151.0,1629.0
1,Location_10,1271.0,503.5,294.0,313.0,1189.0,587.0,407.0,525.0,765.5,...,2814.0,3092.0,2516.0,2238.0,3624.0,3950.0,2209.5,3235.5,4220.0,2957.0
2,Location_100,770.0,463.5,388.5,923.0,895.5,414.0,601.0,458.0,503.0,...,3629.5,3012.0,4081.5,3508.0,3399.0,3298.0,3782.5,3913.0,3808.5,4285.0
3,Location_101,1337.0,738.5,415.5,623.5,712.0,370.0,438.0,715.0,847.0,...,2960.0,3718.0,2804.0,4075.0,4192.5,3368.0,3170.0,4188.5,4395.0,1041.0
4,Location_102,860.0,459.0,491.0,945.0,321.0,384.0,499.0,419.0,422.0,...,4705.0,4994.5,1680.0,3952.0,4187.0,3696.0,3882.0,3694.0,4287.0,4536.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,Location_95,494.0,352.5,264.5,655.5,711.0,1385.0,1344.0,1182.5,1085.5,...,3111.0,3005.0,2331.5,3290.0,3625.0,2708.0,3005.0,2822.0,2776.0,2660.0
312,Location_96,1637.0,346.0,198.5,422.0,595.0,784.0,768.0,714.0,1048.5,...,3148.5,2969.0,2092.0,2391.0,2942.0,3675.0,3949.5,2736.0,3315.5,2765.5
313,Location_97,1848.0,673.5,606.0,811.0,1021.0,655.0,644.0,677.5,1296.0,...,2993.5,4234.0,3355.0,2433.0,3615.0,3374.5,3037.0,3860.0,3452.0,4387.0
314,Location_98,4805.0,627.0,442.5,1901.5,483.0,351.5,611.5,564.5,788.0,...,4422.5,3923.0,3209.0,3999.0,2990.0,2807.0,3264.0,4821.5,4143.5,2984.0


In [29]:
jagung_sen2[jagung_sen2.isna().any(axis=1)]

,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12


### GLCM

In [30]:
jagung_glcm

,location,VH_diss_2024-01,VH_diss_2024-02,VH_diss_2024-03,VH_diss_2024-04,VH_diss_2024-05,VH_diss_2024-06,VH_diss_2024-07,VH_diss_2024-08,VH_diss_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01
0,Location_1,18.722197,18.423864,18.760227,18.640057,20.150341,17.847091,19.019091,17.946364,16.603909,...,5.281976,5.285522,5.288698,5.292312,5.288797,5.277571,5.286350,5.273001,5.301665,5.292399
1,Location_10,21.355000,19.650057,16.289735,14.704602,16.631288,16.115455,14.891318,16.616682,18.066500,...,5.272725,5.281755,5.264791,5.275097,5.270288,5.270308,5.264588,5.235786,5.257377,5.274072
2,Location_100,17.287992,19.288920,19.395455,16.691932,17.001364,17.514000,17.717545,16.870636,17.208955,...,5.272192,5.274204,5.278518,5.284030,5.286574,5.267184,5.274653,5.298937,5.287916,5.287033
3,Location_101,18.136477,19.675341,15.832159,15.280966,17.790455,16.066318,15.448818,16.134409,17.293773,...,5.262316,5.240686,5.257498,5.249365,5.250259,5.263861,5.267834,5.250131,5.268214,5.283510
4,Location_102,17.795265,16.468239,17.121515,17.293864,17.850455,16.877227,17.948727,18.568727,17.203591,...,5.266075,5.281755,5.276442,5.249914,5.275120,5.277261,5.263594,5.254420,5.260797,5.264305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,Location_95,21.160379,17.996705,16.242689,15.863864,17.414621,17.092818,15.799455,16.779727,16.304045,...,5.286155,5.262885,5.272590,5.271856,5.281861,5.264432,5.233698,5.279088,5.285180,5.296767
312,Location_96,18.356288,20.406989,16.807614,16.365341,18.388409,17.703636,17.418545,18.683182,16.703909,...,5.264772,5.263000,5.276250,5.275535,5.268195,5.278662,5.269313,5.274407,5.250058,5.289184
313,Location_97,21.563902,22.442670,21.174848,20.846364,22.670758,23.086318,19.123136,22.443909,19.761682,...,5.299146,5.289028,5.293339,5.301088,5.301301,5.285891,5.290756,5.299894,5.303220,5.298533
314,Location_98,15.518864,17.208580,17.851742,17.722614,16.867386,15.858818,16.386818,19.560182,17.345091,...,5.280041,5.275451,5.269959,5.280002,5.279035,5.285380,5.281542,5.267369,5.256079,5.263852


In [31]:
jagung_glcm[jagung_glcm.isna().any(axis=1)]

,location,VH_diss_2024-01,VH_diss_2024-02,VH_diss_2024-03,VH_diss_2024-04,VH_diss_2024-05,VH_diss_2024-06,VH_diss_2024-07,VH_diss_2024-08,VH_diss_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01


## **Others**

In [32]:
others_sen2 = process_sentinel2('others_sen2.csv')
others_sen1 = process_sentinel1_chirps_terrain('others_sen1_chirps_terrain.csv')
others_glcm = process_glcm('others_glcm.csv')

### SEN2

In [33]:
others_sen2

,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12
0,Location_1,685.0,784.5,407.5,2900.0,378.0,413.0,440.0,1543.0,1058.0,...,3192.0,4340.5,2181.5,2228.0,3919.0,3130.5,2148.0,4122.0,3762.0,4652.0
1,Location_10,550.0,721.5,1173.0,240.0,367.0,334.5,510.5,413.0,896.5,...,5070.5,5577.0,2583.0,3139.5,3060.0,2622.0,2769.0,2852.0,1848.5,3881.5
2,Location_100,1332.0,520.5,525.5,398.5,708.5,676.0,644.5,811.0,961.5,...,2912.5,1757.0,2804.5,2674.5,2449.0,2316.0,3167.5,2745.5,2657.0,2132.0
3,Location_101,1332.0,520.5,525.5,398.5,708.5,676.0,644.5,811.0,961.5,...,2912.5,1757.0,2804.5,2674.5,2449.0,2316.0,3167.5,2745.5,2657.0,2132.0
4,Location_102,1201.0,762.5,379.0,588.0,580.0,589.0,603.0,470.0,635.0,...,3149.0,2354.5,4371.0,2796.0,2598.0,3623.0,2961.5,3163.0,4813.0,3180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,Location_95,1805.5,383.5,1169.0,238.5,267.0,536.5,693.0,609.0,418.5,...,4054.5,3708.5,3128.5,2906.5,2034.5,3602.0,2831.5,2545.5,3066.0,NaN
537,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,Location_97,1194.0,352.5,289.5,332.5,301.5,287.0,250.5,1526.5,362.5,...,3396.0,3891.5,2894.5,3039.0,2014.5,4216.5,3010.0,3881.5,3910.5,NaN
539,Location_98,811.0,4402.5,2067.0,435.0,530.0,384.0,747.0,846.0,753.0,...,5453.0,4387.0,4283.0,3928.0,1495.0,1400.5,2703.0,2934.5,3017.5,4742.0


In [34]:
others_sen2[others_sen2.isna().any(axis=1)]

,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12
10,Location_108,1216.0,544.0,351.0,1525.0,359.5,414.0,295.0,387.5,486.0,...,3736.0,4134.0,3254.5,3171.0,2881.0,2276.5,2674.5,3607.0,3826.0,NaN
12,Location_11,2022.0,1003.0,541.0,842.0,573.0,484.0,829.0,607.0,685.5,...,900.0,2063.0,2673.5,2860.5,1235.0,3421.0,4745.5,3858.0,4828.0,NaN
14,Location_111,3829.5,1134.0,568.5,2767.5,299.5,605.5,473.5,692.5,640.5,...,3235.5,5564.5,2677.5,4302.0,4920.0,4568.0,4091.5,1244.0,3528.5,NaN
15,Location_112,3829.5,1134.0,568.5,2767.5,299.5,605.5,473.5,692.5,640.5,...,3235.5,5564.5,2677.5,4302.0,4920.0,4568.0,4091.5,1244.0,3528.5,NaN
17,Location_114,966.0,284.0,640.0,437.0,985.0,919.5,434.0,405.0,430.0,...,4777.5,3736.0,3585.0,2293.5,4431.0,3808.0,3684.0,3886.0,4735.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,Location_94,1225.0,402.5,304.0,276.5,289.5,323.5,366.0,308.0,407.5,...,2895.0,2915.5,2961.5,2803.0,2713.5,2619.5,2634.5,2783.0,3062.0,NaN
536,Location_95,1805.5,383.5,1169.0,238.5,267.0,536.5,693.0,609.0,418.5,...,4054.5,3708.5,3128.5,2906.5,2034.5,3602.0,2831.5,2545.5,3066.0,NaN
537,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,Location_97,1194.0,352.5,289.5,332.5,301.5,287.0,250.5,1526.5,362.5,...,3396.0,3891.5,2894.5,3039.0,2014.5,4216.5,3010.0,3881.5,3910.5,NaN


In [35]:
def fill_nan_by_group(df):
    # Buat dictionary untuk mengelompokkan kolom berdasarkan prefix (misalnya 'b1', 'b2', ...)
    grouped_columns = {}

    for col in df.columns:
        prefix = ''.join(filter(str.isalpha, col))  # Ambil huruf dari nama kolom (misal 'b1' dari 'b1_2024-01')
        if prefix not in grouped_columns:
            grouped_columns[prefix] = []
        grouped_columns[prefix].append(col)

    # Iterasi setiap grup untuk melakukan pengisian NaN
    for group, cols in grouped_columns.items():
        df[cols] = df[cols].apply(lambda series: series.interpolate(method='linear', limit_direction='both'), axis=0)

    return df

# Terapkan ke dataset
others_sen2 = fill_nan_by_group(others_sen2)

# Tampilkan baris yang masih memiliki NaN (jika ada)
others_sen2[others_sen2.isna().any(axis=1)]

<ipython-input-35-c871d8f62d3c>:13: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[cols] = df[cols].apply(lambda series: series.interpolate(method='linear', limit_direction='both'), axis=0)


,name,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,B8A_2024-03,B8A_2024-04,B8A_2024-05,B8A_2024-06,B8A_2024-07,B8A_2024-08,B8A_2024-09,B8A_2024-10,B8A_2024-11,B8A_2024-12


### SEN 1 ETC

In [36]:
others_sen1

,Location,VV_2024-01,VV_2024-02,VV_2024-03,VV_2024-04,VV_2024-05,VV_2024-06,VV_2024-07,VV_2024-08,VV_2024-09,...,Slope_2024-04,Slope_2024-05,Slope_2024-06,Slope_2024-07,Slope_2024-08,Slope_2024-09,Slope_2024-10,Slope_2024-11,Slope_2024-12,Slope_2025-01
0,Location_1,-9.392434,-13.158668,-9.607953,-11.362101,-10.481134,-9.100350,-12.283118,-11.335416,-6.197215,...,7.665874,7.665874,7.665874,7.665874,7.665874,7.665874,7.665874,7.665874,7.665874,7.665874
1,Location_10,-7.262294,-6.413580,-7.994889,-7.359346,-5.992695,-7.158109,-7.846813,-9.042884,-7.280848,...,4.934383,4.934383,4.934383,4.934383,4.934383,4.934383,4.934383,4.934383,4.934383,4.934383
2,Location_100,-8.723971,-11.697987,-9.209060,-8.052637,-9.307475,-8.863300,-10.367514,-10.594937,-11.092843,...,0.978672,0.978672,0.978672,0.978672,0.978672,0.978672,0.978672,0.978672,0.978672,0.978672
3,Location_101,-8.723971,-11.697987,-9.209060,-8.052637,-9.307475,-8.863300,-10.367514,-10.594937,-11.092843,...,0.978672,0.978672,0.978672,0.978672,0.978672,0.978672,0.978672,0.978672,0.978672,0.978672
4,Location_102,-14.155561,-13.205921,-11.212698,-11.608812,-9.854951,-12.722927,-12.229049,-10.236862,-14.325565,...,2.251451,2.251451,2.251451,2.251451,2.251451,2.251451,2.251451,2.251451,2.251451,2.251451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,Location_95,-8.607758,-7.916439,-7.842438,-7.902386,-8.321121,-6.160026,-7.497924,-9.447027,-9.013570,...,2.922511,2.922511,2.922511,2.922511,2.922511,2.922511,2.922511,2.922511,2.922511,2.922511
537,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,Location_97,-10.252562,-8.341052,-6.842795,-8.330571,-7.131106,-7.871939,-7.933652,-7.617063,-8.404454,...,2.825832,2.825832,2.825832,2.825832,2.825832,2.825832,2.825832,2.825832,2.825832,2.825832
539,Location_98,-5.143514,-7.027567,-6.291998,-6.081826,-7.004133,-6.940199,-8.206582,-8.183225,-8.328983,...,3.908946,3.908946,3.908946,3.908946,3.908946,3.908946,3.908946,3.908946,3.908946,3.908946


In [37]:
others_sen1[others_sen1.isna().any(axis=1)]

,Location,VV_2024-01,VV_2024-02,VV_2024-03,VV_2024-04,VV_2024-05,VV_2024-06,VV_2024-07,VV_2024-08,VV_2024-09,...,Slope_2024-04,Slope_2024-05,Slope_2024-06,Slope_2024-07,Slope_2024-08,Slope_2024-09,Slope_2024-10,Slope_2024-11,Slope_2024-12,Slope_2025-01
537,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### GLCM

In [38]:
others_glcm

,location,VH_diss_2024-01,VH_diss_2024-02,VH_diss_2024-03,VH_diss_2024-04,VH_diss_2024-05,VH_diss_2024-06,VH_diss_2024-07,VH_diss_2024-08,VH_diss_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01
0,Location_1,20.325758,22.695511,21.674356,22.538239,21.567273,21.216955,21.792273,22.046045,21.549727,...,5.294498,5.297996,5.295692,5.295552,5.297818,5.310559,5.305159,5.305762,5.304264,5.312887
1,Location_10,16.737689,16.071705,16.491250,16.109034,16.194659,17.608773,16.939045,17.471318,17.861955,...,5.286994,5.273271,5.282612,5.280913,5.282501,5.294134,5.272366,5.269129,5.270914,5.276386
2,Location_100,16.884848,16.053182,16.908674,16.323864,16.415152,16.027500,15.798364,16.184545,17.875636,...,5.276096,5.279850,5.290062,5.284925,5.271805,5.284345,5.278269,5.262025,5.267539,5.259207
3,Location_101,16.884848,16.053182,16.908674,16.323864,16.415152,16.027500,15.798364,16.184545,17.875636,...,5.276096,5.279850,5.290062,5.284925,5.271805,5.284345,5.278269,5.262025,5.267539,5.259207
4,Location_102,19.781136,16.496989,16.332311,20.319886,16.414356,20.170818,21.263909,14.507545,18.734500,...,5.277203,5.248622,5.278184,5.290080,5.216171,5.269556,5.267172,5.248636,5.281326,5.291090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,Location_95,16.106733,16.002922,15.954886,16.112370,15.574801,15.101023,16.648608,15.648571,16.317614,...,5.252909,5.266110,5.263682,5.261335,5.255411,5.254891,5.262867,5.270077,5.265572,5.253655
537,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,Location_97,17.581250,17.751420,18.636250,16.934886,18.512045,16.624182,18.827091,17.525636,19.187636,...,5.275809,5.275461,5.276725,5.282939,5.272612,5.287899,5.281469,5.275043,5.271693,5.277817
539,Location_98,14.921023,14.362557,14.316402,14.207102,14.505000,15.083955,15.104409,14.262773,14.833500,...,5.257017,5.257969,5.251114,5.271391,5.264619,5.265276,5.247583,5.242492,5.245525,5.264915


In [39]:
others_glcm[others_glcm.isna().any(axis=1)]

,location,VH_diss_2024-01,VH_diss_2024-02,VH_diss_2024-03,VH_diss_2024-04,VH_diss_2024-05,VH_diss_2024-06,VH_diss_2024-07,VH_diss_2024-08,VH_diss_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01
537,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN




# **Merge per Komoditas**



## **Tebu**

In [40]:
data_tebu = merge_datasets(tebu_sen2, tebu_sen1, tebu_glcm, 'tebu')
data_tebu

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01,class
0,Location_1,375.5,652.5,1808.0,4173.0,434.5,523.5,808.5,621.5,591.5,...,5.266343,5.249825,5.247753,5.236627,5.261103,5.271101,5.261828,5.243223,5.252348,tebu
1,Location_10,413.0,580.0,318.5,445.0,490.0,261.0,487.5,471.0,548.5,...,5.290195,5.262574,5.261156,5.245557,5.273984,5.253889,5.266538,5.273718,5.274358,tebu
2,Location_100,877.0,484.5,723.0,618.0,484.0,1515.0,513.0,744.0,432.0,...,5.260171,5.252504,5.254760,5.281956,5.247375,5.267330,5.235875,5.291425,5.269675,tebu
3,Location_101,1067.0,646.0,2348.0,445.0,1026.5,694.0,733.5,1995.0,793.0,...,5.281692,5.280640,5.274906,5.293938,5.282001,5.287665,5.255166,5.283386,5.272209,tebu
4,Location_102,1614.0,782.0,497.5,349.0,598.0,512.0,584.0,810.0,474.0,...,5.285950,5.269042,5.267692,5.256470,5.264337,5.274964,5.274470,5.271800,5.290775,tebu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Location_95,1201.0,1194.0,1405.0,1052.0,1881.5,543.0,786.0,1836.5,726.0,...,5.251318,5.253960,5.272335,5.257065,5.250710,5.253939,5.252784,5.259103,5.273824,tebu
137,Location_96,4004.0,735.5,803.5,465.0,396.0,545.0,793.0,577.0,561.0,...,5.278998,5.277447,5.272930,5.256900,5.265709,5.277344,5.257590,5.273260,5.270668,tebu
138,Location_97,2268.0,710.0,2063.0,854.0,807.5,722.0,844.0,2110.0,642.0,...,5.280895,5.261339,5.268494,5.273131,5.255169,5.258499,5.274448,5.261112,5.282638,tebu
139,Location_98,973.0,681.0,1790.0,718.0,759.0,776.5,1869.5,775.0,1188.5,...,5.264489,5.255463,5.259280,5.273817,5.243448,5.231638,5.226259,5.256745,5.243032,tebu


In [41]:
data_tebu.drop(['VV_2025-01', 'VH_2025-01', 'CHIRPS_2025-01',
                'Aspect_2025-01', 'Elevation_2025-01', 'Slope_2025-01',
                'VH_diss_2025-01', 'VV_asm_2025-01', 'VV_contrast_2025-01',
                'VV_corr_2025-01', 'VV_ent_2025-01'], axis=1, inplace=True)

data_tebu

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
0,Location_1,375.5,652.5,1808.0,4173.0,434.5,523.5,808.5,621.5,591.5,...,5.238523,5.266343,5.249825,5.247753,5.236627,5.261103,5.271101,5.261828,5.243223,tebu
1,Location_10,413.0,580.0,318.5,445.0,490.0,261.0,487.5,471.0,548.5,...,5.279375,5.290195,5.262574,5.261156,5.245557,5.273984,5.253889,5.266538,5.273718,tebu
2,Location_100,877.0,484.5,723.0,618.0,484.0,1515.0,513.0,744.0,432.0,...,5.262306,5.260171,5.252504,5.254760,5.281956,5.247375,5.267330,5.235875,5.291425,tebu
3,Location_101,1067.0,646.0,2348.0,445.0,1026.5,694.0,733.5,1995.0,793.0,...,5.290820,5.281692,5.280640,5.274906,5.293938,5.282001,5.287665,5.255166,5.283386,tebu
4,Location_102,1614.0,782.0,497.5,349.0,598.0,512.0,584.0,810.0,474.0,...,5.279391,5.285950,5.269042,5.267692,5.256470,5.264337,5.274964,5.274470,5.271800,tebu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Location_95,1201.0,1194.0,1405.0,1052.0,1881.5,543.0,786.0,1836.5,726.0,...,5.261040,5.251318,5.253960,5.272335,5.257065,5.250710,5.253939,5.252784,5.259103,tebu
137,Location_96,4004.0,735.5,803.5,465.0,396.0,545.0,793.0,577.0,561.0,...,5.284255,5.278998,5.277447,5.272930,5.256900,5.265709,5.277344,5.257590,5.273260,tebu
138,Location_97,2268.0,710.0,2063.0,854.0,807.5,722.0,844.0,2110.0,642.0,...,5.269207,5.280895,5.261339,5.268494,5.273131,5.255169,5.258499,5.274448,5.261112,tebu
139,Location_98,973.0,681.0,1790.0,718.0,759.0,776.5,1869.5,775.0,1188.5,...,5.243550,5.264489,5.255463,5.259280,5.273817,5.243448,5.231638,5.226259,5.256745,tebu


In [42]:
data_tebu.isna().sum()

,0
location,0
B1_2024-01,0
B1_2024-02,0
B1_2024-03,0
B1_2024-04,0
...,...
VV_ent_2024-09,0
VV_ent_2024-10,0
VV_ent_2024-11,0
VV_ent_2024-12,0


In [43]:
#print data yang mengandung nan saja

data_tebu[data_tebu.isna().any(axis=1)]

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class


## **Padi**

In [44]:
data_padi = merge_datasets(padi_sen2, padi_sen1, padi_glcm, 'padi')
data_padi

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01,class
0,Location_1,1022.0,548.0,245.5,601.0,502.5,547.0,492.0,379.0,659.0,...,5.267448,5.236203,5.252861,5.254998,5.252413,5.253070,5.230378,5.232962,5.284922,padi
1,Location_10,235.5,581.5,367.0,107.0,205.0,306.5,292.0,316.0,425.0,...,5.257500,5.255708,5.253817,5.250058,5.255002,5.229077,5.261958,5.261547,5.252367,padi
2,Location_100,5496.5,3543.0,452.0,488.0,545.0,432.0,443.0,667.0,736.5,...,5.298460,5.257171,5.261669,5.272243,5.275870,5.279690,5.267056,5.273107,5.264176,padi
3,Location_101,5505.5,2093.0,712.0,513.0,515.0,426.0,447.0,619.0,601.5,...,5.300666,5.282329,5.283011,5.285661,5.296321,5.311079,5.309027,5.311099,5.310650,padi
4,Location_102,4667.0,2066.5,3955.0,525.0,423.5,416.5,665.0,771.0,779.0,...,5.278748,5.271373,5.272000,5.271573,5.283117,5.300721,5.292780,5.273093,5.288397,padi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Location_95,3699.0,332.5,1401.0,592.5,495.5,478.5,572.5,746.0,797.5,...,5.279820,5.290793,5.285480,5.286758,5.281129,5.283475,5.295009,5.292830,5.290518,padi
356,Location_96,5650.0,556.0,793.0,337.0,355.5,392.0,551.0,701.0,551.0,...,5.280183,5.238085,5.259750,5.243627,5.284490,5.261643,5.291534,5.281970,5.267442,padi
357,Location_97,5286.5,407.0,818.0,481.0,410.5,429.0,571.5,734.0,734.0,...,5.265829,5.262952,5.259913,5.264276,5.277587,5.243198,5.293644,5.272130,5.263143,padi
358,Location_98,3289.0,445.0,639.0,642.5,417.0,514.0,620.5,569.5,521.5,...,5.296970,5.288000,5.284948,5.278363,5.279864,5.295619,5.288298,5.276369,5.294733,padi


In [45]:
data_padi.drop(['VV_2025-01', 'VH_2025-01', 'CHIRPS_2025-01',
                'Aspect_2025-01', 'Elevation_2025-01', 'Slope_2025-01',
                'VH_diss_2025-01', 'VV_asm_2025-01', 'VV_contrast_2025-01',
                'VV_corr_2025-01', 'VV_ent_2025-01'], axis=1, inplace=True)
data_padi

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
0,Location_1,1022.0,548.0,245.5,601.0,502.5,547.0,492.0,379.0,659.0,...,5.237976,5.267448,5.236203,5.252861,5.254998,5.252413,5.253070,5.230378,5.232962,padi
1,Location_10,235.5,581.5,367.0,107.0,205.0,306.5,292.0,316.0,425.0,...,5.239840,5.257500,5.255708,5.253817,5.250058,5.255002,5.229077,5.261958,5.261547,padi
2,Location_100,5496.5,3543.0,452.0,488.0,545.0,432.0,443.0,667.0,736.5,...,5.287934,5.298460,5.257171,5.261669,5.272243,5.275870,5.279690,5.267056,5.273107,padi
3,Location_101,5505.5,2093.0,712.0,513.0,515.0,426.0,447.0,619.0,601.5,...,5.314700,5.300666,5.282329,5.283011,5.285661,5.296321,5.311079,5.309027,5.311099,padi
4,Location_102,4667.0,2066.5,3955.0,525.0,423.5,416.5,665.0,771.0,779.0,...,5.298946,5.278748,5.271373,5.272000,5.271573,5.283117,5.300721,5.292780,5.273093,padi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Location_95,3699.0,332.5,1401.0,592.5,495.5,478.5,572.5,746.0,797.5,...,5.287816,5.279820,5.290793,5.285480,5.286758,5.281129,5.283475,5.295009,5.292830,padi
356,Location_96,5650.0,556.0,793.0,337.0,355.5,392.0,551.0,701.0,551.0,...,5.295719,5.280183,5.238085,5.259750,5.243627,5.284490,5.261643,5.291534,5.281970,padi
357,Location_97,5286.5,407.0,818.0,481.0,410.5,429.0,571.5,734.0,734.0,...,5.303066,5.265829,5.262952,5.259913,5.264276,5.277587,5.243198,5.293644,5.272130,padi
358,Location_98,3289.0,445.0,639.0,642.5,417.0,514.0,620.5,569.5,521.5,...,5.306455,5.296970,5.288000,5.284948,5.278363,5.279864,5.295619,5.288298,5.276369,padi


In [46]:
#print data yang mengandung nan saja

data_padi[data_padi.isna().any(axis=1)]

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
2,Location_100,5496.5,3543.0,452.0,488.0,545.0,432.0,443.0,667.0,736.5,...,5.287934,5.298460,5.257171,5.261669,5.272243,5.275870,5.279690,5.267056,5.273107,padi
3,Location_101,5505.5,2093.0,712.0,513.0,515.0,426.0,447.0,619.0,601.5,...,5.314700,5.300666,5.282329,5.283011,5.285661,5.296321,5.311079,5.309027,5.311099,padi
4,Location_102,4667.0,2066.5,3955.0,525.0,423.5,416.5,665.0,771.0,779.0,...,5.298946,5.278748,5.271373,5.272000,5.271573,5.283117,5.300721,5.292780,5.273093,padi
5,Location_103,4784.0,812.0,1455.0,569.0,491.5,529.0,749.0,697.5,773.5,...,5.263784,5.269951,5.268738,5.281977,5.258056,5.248530,5.283949,5.282937,5.278126,padi
6,Location_104,3850.5,788.0,632.0,608.5,398.0,403.0,604.0,668.0,509.0,...,5.278624,5.260025,5.227294,5.252391,5.263460,5.258838,5.278103,5.296417,5.279066,padi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Location_95,3699.0,332.5,1401.0,592.5,495.5,478.5,572.5,746.0,797.5,...,5.287816,5.279820,5.290793,5.285480,5.286758,5.281129,5.283475,5.295009,5.292830,padi
356,Location_96,5650.0,556.0,793.0,337.0,355.5,392.0,551.0,701.0,551.0,...,5.295719,5.280183,5.238085,5.259750,5.243627,5.284490,5.261643,5.291534,5.281970,padi
357,Location_97,5286.5,407.0,818.0,481.0,410.5,429.0,571.5,734.0,734.0,...,5.303066,5.265829,5.262952,5.259913,5.264276,5.277587,5.243198,5.293644,5.272130,padi
358,Location_98,3289.0,445.0,639.0,642.5,417.0,514.0,620.5,569.5,521.5,...,5.306455,5.296970,5.288000,5.284948,5.278363,5.279864,5.295619,5.288298,5.276369,padi


In [47]:
data_padi.isna().sum()

,0
location,0
B1_2024-01,0
B1_2024-02,0
B1_2024-03,0
B1_2024-04,0
...,...
VV_ent_2024-09,1
VV_ent_2024-10,1
VV_ent_2024-11,1
VV_ent_2024-12,1


## **Jagung**

In [48]:
jagung_sen2 = process_sentinel2('jagung_sen2.csv')
jagung_sen1 = process_sentinel1_chirps_terrain('jagung_sen1_chirps_terrain.csv')
jagung_glcm = process_glcm('jagung_glcm.csv')

data_jagung = merge_datasets(jagung_sen2, jagung_sen1, jagung_glcm, 'jagung')
data_jagung

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01,class
0,Location_1,344.0,799.0,548.0,1606.0,382.5,284.0,477.0,315.0,1117.0,...,5.285522,5.288698,5.292312,5.288797,5.277571,5.286350,5.273001,5.301665,5.292399,jagung
1,Location_10,1271.0,503.5,294.0,313.0,1189.0,587.0,407.0,525.0,765.5,...,5.281755,5.264791,5.275097,5.270288,5.270308,5.264588,5.235786,5.257377,5.274072,jagung
2,Location_100,770.0,463.5,388.5,923.0,895.5,414.0,601.0,458.0,503.0,...,5.274204,5.278518,5.284030,5.286574,5.267184,5.274653,5.298937,5.287916,5.287033,jagung
3,Location_101,1337.0,738.5,415.5,623.5,712.0,370.0,438.0,715.0,847.0,...,5.240686,5.257498,5.249365,5.250259,5.263861,5.267834,5.250131,5.268214,5.283510,jagung
4,Location_102,860.0,459.0,491.0,945.0,321.0,384.0,499.0,419.0,422.0,...,5.281755,5.276442,5.249914,5.275120,5.277261,5.263594,5.254420,5.260797,5.264305,jagung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,Location_95,494.0,352.5,264.5,655.5,711.0,1385.0,1344.0,1182.5,1085.5,...,5.262885,5.272590,5.271856,5.281861,5.264432,5.233698,5.279088,5.285180,5.296767,jagung
312,Location_96,1637.0,346.0,198.5,422.0,595.0,784.0,768.0,714.0,1048.5,...,5.263000,5.276250,5.275535,5.268195,5.278662,5.269313,5.274407,5.250058,5.289184,jagung
313,Location_97,1848.0,673.5,606.0,811.0,1021.0,655.0,644.0,677.5,1296.0,...,5.289028,5.293339,5.301088,5.301301,5.285891,5.290756,5.299894,5.303220,5.298533,jagung
314,Location_98,4805.0,627.0,442.5,1901.5,483.0,351.5,611.5,564.5,788.0,...,5.275451,5.269959,5.280002,5.279035,5.285380,5.281542,5.267369,5.256079,5.263852,jagung


In [49]:
data_jagung.drop(['VV_2025-01', 'VH_2025-01', 'CHIRPS_2025-01',
                'Aspect_2025-01', 'Elevation_2025-01', 'Slope_2025-01',
                'VH_diss_2025-01', 'VV_asm_2025-01', 'VV_contrast_2025-01',
                'VV_corr_2025-01', 'VV_ent_2025-01'], axis=1, inplace=True)

data_jagung

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
0,Location_1,344.0,799.0,548.0,1606.0,382.5,284.0,477.0,315.0,1117.0,...,5.281976,5.285522,5.288698,5.292312,5.288797,5.277571,5.286350,5.273001,5.301665,jagung
1,Location_10,1271.0,503.5,294.0,313.0,1189.0,587.0,407.0,525.0,765.5,...,5.272725,5.281755,5.264791,5.275097,5.270288,5.270308,5.264588,5.235786,5.257377,jagung
2,Location_100,770.0,463.5,388.5,923.0,895.5,414.0,601.0,458.0,503.0,...,5.272192,5.274204,5.278518,5.284030,5.286574,5.267184,5.274653,5.298937,5.287916,jagung
3,Location_101,1337.0,738.5,415.5,623.5,712.0,370.0,438.0,715.0,847.0,...,5.262316,5.240686,5.257498,5.249365,5.250259,5.263861,5.267834,5.250131,5.268214,jagung
4,Location_102,860.0,459.0,491.0,945.0,321.0,384.0,499.0,419.0,422.0,...,5.266075,5.281755,5.276442,5.249914,5.275120,5.277261,5.263594,5.254420,5.260797,jagung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,Location_95,494.0,352.5,264.5,655.5,711.0,1385.0,1344.0,1182.5,1085.5,...,5.286155,5.262885,5.272590,5.271856,5.281861,5.264432,5.233698,5.279088,5.285180,jagung
312,Location_96,1637.0,346.0,198.5,422.0,595.0,784.0,768.0,714.0,1048.5,...,5.264772,5.263000,5.276250,5.275535,5.268195,5.278662,5.269313,5.274407,5.250058,jagung
313,Location_97,1848.0,673.5,606.0,811.0,1021.0,655.0,644.0,677.5,1296.0,...,5.299146,5.289028,5.293339,5.301088,5.301301,5.285891,5.290756,5.299894,5.303220,jagung
314,Location_98,4805.0,627.0,442.5,1901.5,483.0,351.5,611.5,564.5,788.0,...,5.280041,5.275451,5.269959,5.280002,5.279035,5.285380,5.281542,5.267369,5.256079,jagung


In [50]:
data_jagung.isna().sum()

,0
location,0
B1_2024-01,0
B1_2024-02,0
B1_2024-03,0
B1_2024-04,8
...,...
VV_ent_2024-09,0
VV_ent_2024-10,0
VV_ent_2024-11,0
VV_ent_2024-12,0


In [51]:
#print data yang mengandung nan saja

data_jagung[data_jagung.isna().any(axis=1)]

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
1,Location_10,1271.0,503.5,294.0,313.0,1189.0,587.0,407.0,525.0,765.5,...,5.272725,5.281755,5.264791,5.275097,5.270288,5.270308,5.264588,5.235786,5.257377,jagung
7,Location_105,1072.0,597.5,498.5,708.0,709.5,679.0,760.0,1162.0,610.0,...,5.282514,5.283459,5.266667,5.269384,5.277688,5.279656,5.269032,5.263624,5.270233,jagung
8,Location_106,1023.0,431.5,253.5,649.0,803.0,564.0,448.0,570.0,730.0,...,5.264628,5.274583,5.272431,5.263800,5.246410,5.266933,5.242674,5.249290,5.238394,jagung
23,Location_12,1338.0,385.5,3217.0,616.5,394.5,379.0,374.0,511.5,467.0,...,5.300591,5.265739,5.255763,5.263075,5.270665,5.279147,5.272206,5.287494,5.275256,jagung
44,Location_139,1161.0,529.0,325.0,641.0,480.0,532.0,503.0,394.0,499.5,...,5.271453,5.282744,5.288600,5.299500,5.282910,5.271119,5.280667,5.289969,5.292272,jagung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,Location_69,875.0,617.5,426.0,NaN,659.0,732.5,572.0,494.5,821.5,...,5.294656,5.285514,5.291997,5.294197,5.281617,5.298788,5.294074,5.286088,5.283425,jagung
291,Location_77,1005.0,427.5,321.0,933.0,679.5,395.0,450.5,571.0,679.5,...,5.264461,5.283137,5.259565,5.279323,5.281617,5.258322,5.257077,5.270990,5.268316,jagung
292,Location_78,881.0,556.5,465.5,437.0,350.5,655.0,570.0,486.0,409.0,...,5.297413,5.277802,5.265506,5.272949,5.294883,5.277983,5.263504,5.273676,5.293136,jagung
307,Location_91,1295.0,1450.0,473.0,736.5,371.5,410.0,357.0,335.0,327.0,...,5.283034,5.281646,5.272651,5.266931,5.282501,5.277833,5.288801,5.276845,5.266109,jagung


## **Others**

In [52]:
others_sen2 = process_sentinel2('others_sen2.csv')
others_sen1 = process_sentinel1_chirps_terrain('others_sen1_chirps_terrain.csv')
others_glcm = process_glcm('others_glcm.csv')

data_others = merge_datasets(others_sen2, others_sen1, others_glcm, 'others')
data_others

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,VV_ent_2025-01,class
0,Location_1,685.0,784.5,407.5,2900.0,378.0,413.0,440.0,1543.0,1058.0,...,5.297996,5.295692,5.295552,5.297818,5.310559,5.305159,5.305762,5.304264,5.312887,others
1,Location_10,550.0,721.5,1173.0,240.0,367.0,334.5,510.5,413.0,896.5,...,5.273271,5.282612,5.280913,5.282501,5.294134,5.272366,5.269129,5.270914,5.276386,others
2,Location_100,1332.0,520.5,525.5,398.5,708.5,676.0,644.5,811.0,961.5,...,5.279850,5.290062,5.284925,5.271805,5.284345,5.278269,5.262025,5.267539,5.259207,others
3,Location_101,1332.0,520.5,525.5,398.5,708.5,676.0,644.5,811.0,961.5,...,5.279850,5.290062,5.284925,5.271805,5.284345,5.278269,5.262025,5.267539,5.259207,others
4,Location_102,1201.0,762.5,379.0,588.0,580.0,589.0,603.0,470.0,635.0,...,5.248622,5.278184,5.290080,5.216171,5.269556,5.267172,5.248636,5.281326,5.291090,others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,Location_95,1805.5,383.5,1169.0,238.5,267.0,536.5,693.0,609.0,418.5,...,5.266110,5.263682,5.261335,5.255411,5.254891,5.262867,5.270077,5.265572,5.253655,others
537,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,others
538,Location_97,1194.0,352.5,289.5,332.5,301.5,287.0,250.5,1526.5,362.5,...,5.275461,5.276725,5.282939,5.272612,5.287899,5.281469,5.275043,5.271693,5.277817,others
539,Location_98,811.0,4402.5,2067.0,435.0,530.0,384.0,747.0,846.0,753.0,...,5.257969,5.251114,5.271391,5.264619,5.265276,5.247583,5.242492,5.245525,5.264915,others


In [53]:
data_others.drop(['VV_2025-01', 'VH_2025-01', 'CHIRPS_2025-01',
                'Aspect_2025-01', 'Elevation_2025-01', 'Slope_2025-01',
                'VH_diss_2025-01', 'VV_asm_2025-01', 'VV_contrast_2025-01',
                'VV_corr_2025-01', 'VV_ent_2025-01'], axis=1, inplace=True)

data_others

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
0,Location_1,685.0,784.5,407.5,2900.0,378.0,413.0,440.0,1543.0,1058.0,...,5.294498,5.297996,5.295692,5.295552,5.297818,5.310559,5.305159,5.305762,5.304264,others
1,Location_10,550.0,721.5,1173.0,240.0,367.0,334.5,510.5,413.0,896.5,...,5.286994,5.273271,5.282612,5.280913,5.282501,5.294134,5.272366,5.269129,5.270914,others
2,Location_100,1332.0,520.5,525.5,398.5,708.5,676.0,644.5,811.0,961.5,...,5.276096,5.279850,5.290062,5.284925,5.271805,5.284345,5.278269,5.262025,5.267539,others
3,Location_101,1332.0,520.5,525.5,398.5,708.5,676.0,644.5,811.0,961.5,...,5.276096,5.279850,5.290062,5.284925,5.271805,5.284345,5.278269,5.262025,5.267539,others
4,Location_102,1201.0,762.5,379.0,588.0,580.0,589.0,603.0,470.0,635.0,...,5.277203,5.248622,5.278184,5.290080,5.216171,5.269556,5.267172,5.248636,5.281326,others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,Location_95,1805.5,383.5,1169.0,238.5,267.0,536.5,693.0,609.0,418.5,...,5.252909,5.266110,5.263682,5.261335,5.255411,5.254891,5.262867,5.270077,5.265572,others
537,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,others
538,Location_97,1194.0,352.5,289.5,332.5,301.5,287.0,250.5,1526.5,362.5,...,5.275809,5.275461,5.276725,5.282939,5.272612,5.287899,5.281469,5.275043,5.271693,others
539,Location_98,811.0,4402.5,2067.0,435.0,530.0,384.0,747.0,846.0,753.0,...,5.257017,5.257969,5.251114,5.271391,5.264619,5.265276,5.247583,5.242492,5.245525,others


In [54]:
#print data yang mengandung nan saja

data_others[data_others.isna().any(axis=1)]

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
10,Location_108,1216.0,544.0,351.0,1525.0,359.5,414.0,295.0,387.5,486.0,...,5.282746,5.276778,5.271612,5.266229,5.268173,5.278769,5.285763,5.271825,5.284299,others
12,Location_11,2022.0,1003.0,541.0,842.0,573.0,484.0,829.0,607.0,685.5,...,5.288028,5.268810,5.273688,5.289755,5.272084,5.263671,5.284065,5.293514,5.281407,others
14,Location_111,3829.5,1134.0,568.5,2767.5,299.5,605.5,473.5,692.5,640.5,...,5.279442,5.269351,5.277751,5.277731,5.280860,5.274881,5.280721,5.277933,5.270948,others
15,Location_112,3829.5,1134.0,568.5,2767.5,299.5,605.5,473.5,692.5,640.5,...,5.279442,5.269351,5.277751,5.277731,5.280860,5.274881,5.280721,5.277933,5.270948,others
17,Location_114,966.0,284.0,640.0,437.0,985.0,919.5,434.0,405.0,430.0,...,5.286970,5.267719,5.275856,5.249819,5.278940,5.294887,5.291514,5.300242,5.290011,others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,Location_94,1225.0,402.5,304.0,276.5,289.5,323.5,366.0,308.0,407.5,...,5.261542,5.253628,5.264632,5.273665,5.252479,5.259427,5.268029,5.265095,5.253094,others
536,Location_95,1805.5,383.5,1169.0,238.5,267.0,536.5,693.0,609.0,418.5,...,5.252909,5.266110,5.263682,5.261335,5.255411,5.254891,5.262867,5.270077,5.265572,others
537,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,others
538,Location_97,1194.0,352.5,289.5,332.5,301.5,287.0,250.5,1526.5,362.5,...,5.275809,5.275461,5.276725,5.282939,5.272612,5.287899,5.281469,5.275043,5.271693,others


In [55]:
data_tebu.isna().sum().to_frame().rename(columns={0: 'Jumlah NaN'})

,Jumlah NaN
location,0
B1_2024-01,0
B1_2024-02,0
B1_2024-03,0
B1_2024-04,0
...,...
VV_ent_2024-09,0
VV_ent_2024-10,0
VV_ent_2024-11,0
VV_ent_2024-12,0


In [56]:
# prompt: data_others.isna().sum() tapi terlihat semua

data_others.isna().sum().to_frame().rename(columns={0: 'Jumlah NaN'})

,Jumlah NaN
location,0
B1_2024-01,1
B1_2024-02,1
B1_2024-03,1
B1_2024-04,4
...,...
VV_ent_2024-09,1
VV_ent_2024-10,1
VV_ent_2024-11,1
VV_ent_2024-12,1


In [57]:
data_others.isna().sum()

,0
location,0
B1_2024-01,1
B1_2024-02,1
B1_2024-03,1
B1_2024-04,4
...,...
VV_ent_2024-09,1
VV_ent_2024-10,1
VV_ent_2024-11,1
VV_ent_2024-12,1




---------




#**Merge**

In [58]:
print(data_tebu.shape)
print(data_padi.shape)
print(data_jagung.shape)
print(data_others.shape)

(141, 242)
(360, 242)
(316, 242)
(541, 242)


In [59]:
dataset = pd.concat([data_tebu, data_padi, data_jagung, data_others], axis=0, ignore_index=True)
dataset

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
0,Location_1,375.5,652.5,1808.0,4173.0,434.5,523.5,808.5,621.5,591.5,...,5.238523,5.266343,5.249825,5.247753,5.236627,5.261103,5.271101,5.261828,5.243223,tebu
1,Location_10,413.0,580.0,318.5,445.0,490.0,261.0,487.5,471.0,548.5,...,5.279375,5.290195,5.262574,5.261156,5.245557,5.273984,5.253889,5.266538,5.273718,tebu
2,Location_100,877.0,484.5,723.0,618.0,484.0,1515.0,513.0,744.0,432.0,...,5.262306,5.260171,5.252504,5.254760,5.281956,5.247375,5.267330,5.235875,5.291425,tebu
3,Location_101,1067.0,646.0,2348.0,445.0,1026.5,694.0,733.5,1995.0,793.0,...,5.290820,5.281692,5.280640,5.274906,5.293938,5.282001,5.287665,5.255166,5.283386,tebu
4,Location_102,1614.0,782.0,497.5,349.0,598.0,512.0,584.0,810.0,474.0,...,5.279391,5.285950,5.269042,5.267692,5.256470,5.264337,5.274964,5.274470,5.271800,tebu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,Location_95,1805.5,383.5,1169.0,238.5,267.0,536.5,693.0,609.0,418.5,...,5.252909,5.266110,5.263682,5.261335,5.255411,5.254891,5.262867,5.270077,5.265572,others
1354,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,others
1355,Location_97,1194.0,352.5,289.5,332.5,301.5,287.0,250.5,1526.5,362.5,...,5.275809,5.275461,5.276725,5.282939,5.272612,5.287899,5.281469,5.275043,5.271693,others
1356,Location_98,811.0,4402.5,2067.0,435.0,530.0,384.0,747.0,846.0,753.0,...,5.257017,5.257969,5.251114,5.271391,5.264619,5.265276,5.247583,5.242492,5.245525,others


In [60]:
dataset[dataset.isna().any(axis=1)]

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
143,Location_100,5496.5,3543.0,452.0,488.0,545.0,432.0,443.0,667.0,736.5,...,5.287934,5.298460,5.257171,5.261669,5.272243,5.275870,5.279690,5.267056,5.273107,padi
144,Location_101,5505.5,2093.0,712.0,513.0,515.0,426.0,447.0,619.0,601.5,...,5.314700,5.300666,5.282329,5.283011,5.285661,5.296321,5.311079,5.309027,5.311099,padi
145,Location_102,4667.0,2066.5,3955.0,525.0,423.5,416.5,665.0,771.0,779.0,...,5.298946,5.278748,5.271373,5.272000,5.271573,5.283117,5.300721,5.292780,5.273093,padi
146,Location_103,4784.0,812.0,1455.0,569.0,491.5,529.0,749.0,697.5,773.5,...,5.263784,5.269951,5.268738,5.281977,5.258056,5.248530,5.283949,5.282937,5.278126,padi
147,Location_104,3850.5,788.0,632.0,608.5,398.0,403.0,604.0,668.0,509.0,...,5.278624,5.260025,5.227294,5.252391,5.263460,5.258838,5.278103,5.296417,5.279066,padi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,Location_94,1225.0,402.5,304.0,276.5,289.5,323.5,366.0,308.0,407.5,...,5.261542,5.253628,5.264632,5.273665,5.252479,5.259427,5.268029,5.265095,5.253094,others
1353,Location_95,1805.5,383.5,1169.0,238.5,267.0,536.5,693.0,609.0,418.5,...,5.252909,5.266110,5.263682,5.261335,5.255411,5.254891,5.262867,5.270077,5.265572,others
1354,Location_96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,others
1355,Location_97,1194.0,352.5,289.5,332.5,301.5,287.0,250.5,1526.5,362.5,...,5.275809,5.275461,5.276725,5.282939,5.272612,5.287899,5.281469,5.275043,5.271693,others


In [61]:
list(dataset.columns)

['location',
 'B1_2024-01',
 'B1_2024-02',
 'B1_2024-03',
 'B1_2024-04',
 'B1_2024-05',
 'B1_2024-06',
 'B1_2024-07',
 'B1_2024-08',
 'B1_2024-09',
 'B1_2024-10',
 'B1_2024-11',
 'B1_2024-12',
 'B2_2024-01',
 'B2_2024-02',
 'B2_2024-03',
 'B2_2024-04',
 'B2_2024-05',
 'B2_2024-06',
 'B2_2024-07',
 'B2_2024-08',
 'B2_2024-09',
 'B2_2024-10',
 'B2_2024-11',
 'B2_2024-12',
 'B3_2024-01',
 'B3_2024-02',
 'B3_2024-03',
 'B3_2024-04',
 'B3_2024-05',
 'B3_2024-06',
 'B3_2024-07',
 'B3_2024-08',
 'B3_2024-09',
 'B3_2024-10',
 'B3_2024-11',
 'B3_2024-12',
 'B4_2024-01',
 'B4_2024-02',
 'B4_2024-03',
 'B4_2024-04',
 'B4_2024-05',
 'B4_2024-06',
 'B4_2024-07',
 'B4_2024-08',
 'B4_2024-09',
 'B4_2024-10',
 'B4_2024-11',
 'B4_2024-12',
 'B5_2024-01',
 'B5_2024-02',
 'B5_2024-03',
 'B5_2024-04',
 'B5_2024-05',
 'B5_2024-06',
 'B5_2024-07',
 'B5_2024-08',
 'B5_2024-09',
 'B5_2024-10',
 'B5_2024-11',
 'B5_2024-12',
 'B6_2024-01',
 'B6_2024-02',
 'B6_2024-03',
 'B6_2024-04',
 'B6_2024-05',
 'B6_2024-06

In [62]:
cek = dataset.copy()
cek.drop(['location', 'class'], axis=1, inplace=True)
columns = cek.columns
dataset =  dataset.dropna(subset=columns, how='all')
dataset

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
0,Location_1,375.5,652.5,1808.0,4173.0,434.5,523.5,808.5,621.5,591.5,...,5.238523,5.266343,5.249825,5.247753,5.236627,5.261103,5.271101,5.261828,5.243223,tebu
1,Location_10,413.0,580.0,318.5,445.0,490.0,261.0,487.5,471.0,548.5,...,5.279375,5.290195,5.262574,5.261156,5.245557,5.273984,5.253889,5.266538,5.273718,tebu
2,Location_100,877.0,484.5,723.0,618.0,484.0,1515.0,513.0,744.0,432.0,...,5.262306,5.260171,5.252504,5.254760,5.281956,5.247375,5.267330,5.235875,5.291425,tebu
3,Location_101,1067.0,646.0,2348.0,445.0,1026.5,694.0,733.5,1995.0,793.0,...,5.290820,5.281692,5.280640,5.274906,5.293938,5.282001,5.287665,5.255166,5.283386,tebu
4,Location_102,1614.0,782.0,497.5,349.0,598.0,512.0,584.0,810.0,474.0,...,5.279391,5.285950,5.269042,5.267692,5.256470,5.264337,5.274964,5.274470,5.271800,tebu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,Location_94,1225.0,402.5,304.0,276.5,289.5,323.5,366.0,308.0,407.5,...,5.261542,5.253628,5.264632,5.273665,5.252479,5.259427,5.268029,5.265095,5.253094,others
1353,Location_95,1805.5,383.5,1169.0,238.5,267.0,536.5,693.0,609.0,418.5,...,5.252909,5.266110,5.263682,5.261335,5.255411,5.254891,5.262867,5.270077,5.265572,others
1355,Location_97,1194.0,352.5,289.5,332.5,301.5,287.0,250.5,1526.5,362.5,...,5.275809,5.275461,5.276725,5.282939,5.272612,5.287899,5.281469,5.275043,5.271693,others
1356,Location_98,811.0,4402.5,2067.0,435.0,530.0,384.0,747.0,846.0,753.0,...,5.257017,5.257969,5.251114,5.271391,5.264619,5.265276,5.247583,5.242492,5.245525,others


In [63]:
dataset_nan = dataset[dataset[columns].isna().all(axis=1)]
print(dataset_nan)

Empty DataFrame
Columns: [location, B1_2024-01, B1_2024-02, B1_2024-03, B1_2024-04, B1_2024-05, B1_2024-06, B1_2024-07, B1_2024-08, B1_2024-09, B1_2024-10, B1_2024-11, B1_2024-12, B2_2024-01, B2_2024-02, B2_2024-03, B2_2024-04, B2_2024-05, B2_2024-06, B2_2024-07, B2_2024-08, B2_2024-09, B2_2024-10, B2_2024-11, B2_2024-12, B3_2024-01, B3_2024-02, B3_2024-03, B3_2024-04, B3_2024-05, B3_2024-06, B3_2024-07, B3_2024-08, B3_2024-09, B3_2024-10, B3_2024-11, B3_2024-12, B4_2024-01, B4_2024-02, B4_2024-03, B4_2024-04, B4_2024-05, B4_2024-06, B4_2024-07, B4_2024-08, B4_2024-09, B4_2024-10, B4_2024-11, B4_2024-12, B5_2024-01, B5_2024-02, B5_2024-03, B5_2024-04, B5_2024-05, B5_2024-06, B5_2024-07, B5_2024-08, B5_2024-09, B5_2024-10, B5_2024-11, B5_2024-12, B6_2024-01, B6_2024-02, B6_2024-03, B6_2024-04, B6_2024-05, B6_2024-06, B6_2024-07, B6_2024-08, B6_2024-09, B6_2024-10, B6_2024-11, B6_2024-12, B7_2024-01, B7_2024-02, B7_2024-03, B7_2024-04, B7_2024-05, B7_2024-06, B7_2024-07, B7_2024-08, B7_2

In [64]:
dataset[dataset.isna().any(axis=1)]

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
143,Location_100,5496.5,3543.0,452.0,488.0,545.0,432.0,443.0,667.0,736.5,...,5.287934,5.298460,5.257171,5.261669,5.272243,5.275870,5.279690,5.267056,5.273107,padi
144,Location_101,5505.5,2093.0,712.0,513.0,515.0,426.0,447.0,619.0,601.5,...,5.314700,5.300666,5.282329,5.283011,5.285661,5.296321,5.311079,5.309027,5.311099,padi
145,Location_102,4667.0,2066.5,3955.0,525.0,423.5,416.5,665.0,771.0,779.0,...,5.298946,5.278748,5.271373,5.272000,5.271573,5.283117,5.300721,5.292780,5.273093,padi
146,Location_103,4784.0,812.0,1455.0,569.0,491.5,529.0,749.0,697.5,773.5,...,5.263784,5.269951,5.268738,5.281977,5.258056,5.248530,5.283949,5.282937,5.278126,padi
147,Location_104,3850.5,788.0,632.0,608.5,398.0,403.0,604.0,668.0,509.0,...,5.278624,5.260025,5.227294,5.252391,5.263460,5.258838,5.278103,5.296417,5.279066,padi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,Location_93,3046.0,1052.5,2439.0,382.5,318.5,359.0,358.0,3557.5,499.0,...,5.274734,5.270938,5.265492,5.273480,5.274071,5.298838,5.287844,5.290972,5.265632,others
1352,Location_94,1225.0,402.5,304.0,276.5,289.5,323.5,366.0,308.0,407.5,...,5.261542,5.253628,5.264632,5.273665,5.252479,5.259427,5.268029,5.265095,5.253094,others
1353,Location_95,1805.5,383.5,1169.0,238.5,267.0,536.5,693.0,609.0,418.5,...,5.252909,5.266110,5.263682,5.261335,5.255411,5.254891,5.262867,5.270077,5.265572,others
1355,Location_97,1194.0,352.5,289.5,332.5,301.5,287.0,250.5,1526.5,362.5,...,5.275809,5.275461,5.276725,5.282939,5.272612,5.287899,5.281469,5.275043,5.271693,others


In [65]:
#print data yang mengandung nan saja

dataset[dataset.isna().any(axis=1)]

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
143,Location_100,5496.5,3543.0,452.0,488.0,545.0,432.0,443.0,667.0,736.5,...,5.287934,5.298460,5.257171,5.261669,5.272243,5.275870,5.279690,5.267056,5.273107,padi
144,Location_101,5505.5,2093.0,712.0,513.0,515.0,426.0,447.0,619.0,601.5,...,5.314700,5.300666,5.282329,5.283011,5.285661,5.296321,5.311079,5.309027,5.311099,padi
145,Location_102,4667.0,2066.5,3955.0,525.0,423.5,416.5,665.0,771.0,779.0,...,5.298946,5.278748,5.271373,5.272000,5.271573,5.283117,5.300721,5.292780,5.273093,padi
146,Location_103,4784.0,812.0,1455.0,569.0,491.5,529.0,749.0,697.5,773.5,...,5.263784,5.269951,5.268738,5.281977,5.258056,5.248530,5.283949,5.282937,5.278126,padi
147,Location_104,3850.5,788.0,632.0,608.5,398.0,403.0,604.0,668.0,509.0,...,5.278624,5.260025,5.227294,5.252391,5.263460,5.258838,5.278103,5.296417,5.279066,padi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,Location_93,3046.0,1052.5,2439.0,382.5,318.5,359.0,358.0,3557.5,499.0,...,5.274734,5.270938,5.265492,5.273480,5.274071,5.298838,5.287844,5.290972,5.265632,others
1352,Location_94,1225.0,402.5,304.0,276.5,289.5,323.5,366.0,308.0,407.5,...,5.261542,5.253628,5.264632,5.273665,5.252479,5.259427,5.268029,5.265095,5.253094,others
1353,Location_95,1805.5,383.5,1169.0,238.5,267.0,536.5,693.0,609.0,418.5,...,5.252909,5.266110,5.263682,5.261335,5.255411,5.254891,5.262867,5.270077,5.265572,others
1355,Location_97,1194.0,352.5,289.5,332.5,301.5,287.0,250.5,1526.5,362.5,...,5.275809,5.275461,5.276725,5.282939,5.272612,5.287899,5.281469,5.275043,5.271693,others


In [66]:
data_without_nan = dataset.dropna(subset=columns, how='any')
data_without_nan

,location,B1_2024-01,B1_2024-02,B1_2024-03,B1_2024-04,B1_2024-05,B1_2024-06,B1_2024-07,B1_2024-08,B1_2024-09,...,VV_ent_2024-04,VV_ent_2024-05,VV_ent_2024-06,VV_ent_2024-07,VV_ent_2024-08,VV_ent_2024-09,VV_ent_2024-10,VV_ent_2024-11,VV_ent_2024-12,class
0,Location_1,375.5,652.5,1808.0,4173.0,434.5,523.5,808.5,621.5,591.5,...,5.238523,5.266343,5.249825,5.247753,5.236627,5.261103,5.271101,5.261828,5.243223,tebu
1,Location_10,413.0,580.0,318.5,445.0,490.0,261.0,487.5,471.0,548.5,...,5.279375,5.290195,5.262574,5.261156,5.245557,5.273984,5.253889,5.266538,5.273718,tebu
2,Location_100,877.0,484.5,723.0,618.0,484.0,1515.0,513.0,744.0,432.0,...,5.262306,5.260171,5.252504,5.254760,5.281956,5.247375,5.267330,5.235875,5.291425,tebu
3,Location_101,1067.0,646.0,2348.0,445.0,1026.5,694.0,733.5,1995.0,793.0,...,5.290820,5.281692,5.280640,5.274906,5.293938,5.282001,5.287665,5.255166,5.283386,tebu
4,Location_102,1614.0,782.0,497.5,349.0,598.0,512.0,584.0,810.0,474.0,...,5.279391,5.285950,5.269042,5.267692,5.256470,5.264337,5.274964,5.274470,5.271800,tebu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344,Location_87,729.0,405.0,345.0,416.0,869.5,578.5,411.0,804.5,767.5,...,5.239265,5.252907,5.280079,5.276531,5.264420,5.259160,5.273824,5.253396,5.252046,others
1345,Location_88,1280.0,799.5,570.5,592.5,633.0,925.5,837.0,1077.0,1865.0,...,5.279066,5.274812,5.288281,5.277521,5.272784,5.270111,5.292791,5.276701,5.278095,others
1346,Location_89,1280.0,799.5,570.5,592.5,633.0,925.5,837.0,1077.0,1865.0,...,5.279066,5.274812,5.288281,5.277521,5.272784,5.270111,5.292791,5.276701,5.278095,others
1347,Location_9,1449.5,394.0,447.5,1666.5,1197.0,491.0,450.5,418.5,452.5,...,5.277642,5.254273,5.275746,5.278970,5.280733,5.281858,5.266729,5.268897,5.274100,others


In [67]:
data_without_nan.to_csv('data_without_nan.csv', index=False)